In [2]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
import warnings
warnings.filterwarnings("ignore")
ex4 = pd.read_stata(r"../../data/Ex4.dta")
data  = ex4[["K","L","Y","CO2"]]
K=2
L=1
M=1
xref=data.iloc[:,0:K]
yref=data.iloc[:,K:K+L]
bref=data.iloc[:,K+L:K+L+M]
x=data.iloc[0,0:K]
y=data.iloc[0,K:K+L]
b=data.iloc[0,K+L:K+L+M]
model = ConcreteModel()
model.I = Set(initialize = range(xref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
model.K = Set(initialize = range(len(x))) # 采用列表初始化投入变量个数的集合
model.L = Set(initialize = range(len(y))) # 采用列表初始化产出变量个数的集合
model.M = Set(initialize = range(len(b))) # 采用列表初始化产出变量个数的集合
model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
def objective_rule(model):
    """Return the proper objective function"""
    return model.pomega*1
def first_rule(model, i):
    """Return the proper input constraint"""
    return -sum(model.px[k] * xref.iloc[i,k] for k in model.K
            ) + sum(model.py[l] * yref.iloc[i,l] for l in model.L
            ) - sum(model.pb[m] * bref.iloc[i,m] for m in model.M)    <= 0
def second_rule(model,k):
    """Return the proper second constraint"""
    return -model.px[k] <= -1 / (len(model.K)*x[k] )
def third_rule(model,l):
    """Return the proper third constraint"""
    return model.pomega / (len(model.L)+len(model.M)) /y[l]-  model.py[l]<=0
def forth_rule(model,m):
    """Return the proper forth constraint"""
    return model.pomega / (len(model.L)+len(model.M)) /b[m] - model.pb[m]<=0
def fifth_rule(model):
    """Return the proper fifth constraint"""
    return model.pomega + sum(model.px[k] * x[k] for k in model.K
            ) - sum(model.py[l] * y[l] for l in model.L
            ) + sum(model.pb[m] * b[m] for m in model.M)<=1                           
model.obj = Objective(rule=objective_rule, sense=maximize, doc='objective function')
model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')
model.fifth = Constraint(         rule= fifth_rule , doc='fifth constraint')             
opt = SolverFactory('mosek') # 指定 mosek 作为求解器
solution = opt.solve(model) # 调用求解器求解
px = np.asarray(list(model.px[:].value))
py = np.asarray(list(model.py[:].value))
pb = np.asarray(list(model.pb[:].value))
obj = value(model.obj) 
print("optimum shadow price x: \n {} ".format(px))
print("optimum shadow price y: \n {} ".format(py))
print("optimum shadow price b: \n {} ".format(pb))
print("optimal object: {}".format(obj))

optimum shadow price x: 
 [1.08261723e-05 1.16934447e-04] 
optimum shadow price y: 
 [2.99116634e-05] 
optimum shadow price b: 
 [3.95856639e-10] 
optimal object: 0.29213054708939895


In [3]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
ex4 = pd.read_stata(r"../../data/Ex4.dta")
data  = ex4[["K","L","Y","CO2"]]
K=2
L=1
M=1
xref=data.iloc[:,0:K]
yref=data.iloc[:,K:K+L]
bref=data.iloc[:,K+L:K+L+M]
px,py,pb,obj = {},{},{},{}             
for j in range(data.shape[0]):
    x=data.iloc[j,0:K]
    y=data.iloc[j,K:K+L]
    b=data.iloc[j,K+L:K+L+M]
    model = ConcreteModel()
    model.I = Set(initialize = range(xref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
    model.K = Set(initialize = range(len(x))) # 采用列表初始化投入变量个数的集合
    model.L = Set(initialize = range(len(y))) # 采用列表初始化产出变量个数的集合
    model.M = Set(initialize = range(len(b))) # 采用列表初始化产出变量个数的集合
    model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
    model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
    model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
    model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
    def objective_rule(model):
        """Return the proper objective function"""
        return model.pomega*1
    def first_rule(model, i):
        """Return the proper input constraint"""
        return -sum(model.px[k] * xref.iloc[i,k] for k in model.K
                ) + sum(model.py[l] * yref.iloc[i,l] for l in model.L
                ) - sum(model.pb[m] * bref.iloc[i,m] for m in model.M)    <= 0
    def second_rule(model,k):
        """Return the proper second constraint"""
        return -model.px[k] <= -1 / (len(model.K)*x[k] )
    def third_rule(model,l):
        """Return the proper third constraint"""
        return model.pomega / (len(model.L)+len(model.M)) /y[l]-  model.py[l]<=0
    def forth_rule(model,m):
        """Return the proper forth constraint"""
        return model.pomega / (len(model.L)+len(model.M)) /b[m] - model.pb[m]<=0
    def fifth_rule(model):
        """Return the proper fifth constraint"""
        return model.pomega + sum(model.px[k] * x[k] for k in model.K
                ) - sum(model.py[l] * y[l] for l in model.L
                ) + sum(model.pb[m] * b[m] for m in model.M)<=1                           
    model.obj = Objective(rule=objective_rule, sense=maximize, doc='objective function')
    model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
    model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
    model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
    model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')
    model.fifth = Constraint(         rule= fifth_rule , doc='fifth constraint')             
    opt = SolverFactory('mosek') # 指定 mosek 作为求解器
    solution = opt.solve(model) # 调用求解器求解
    if solution.solver.termination_condition == "optimal":     # 终止条件 一般包括三种 optimal, feasible, infeasible
        px[j] = np.asarray(list(model.px[:].value))
        py[j] = np.asarray(list(model.py[:].value))
        pb[j] = np.asarray(list(model.pb[:].value))
        obj[j] = value(model.obj) 

In [4]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def sbmdual( dataframe, varname ,P,Q):
    """
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    varname: 投入变量、期望产出变量和非期望产出变量的列表，如["K","L","Y","CO2"]。
    P: 投入变量的个数。
    Q:期望产出变量的个数。
    """
    data  = dataframe.loc[:,varname]
    K=P
    L=Q
    M=data.shape[1]-P-Q
    xref=data.iloc[:,0:K]
    yref=data.iloc[:,K:K+L]
    bref=data.iloc[:,K+L:K+L+M]
    xcol=varname[0:K]
    ycol=varname[K:K+L]
    bcol=varname[K+L:K+L+M]
    px,py,pb,obj = {},{},{},{}             
    for j in range(data.shape[0]):
        x=data.iloc[j,0:K]
        y=data.iloc[j,K:K+L]
        b=data.iloc[j,K+L:K+L+M]
        model = ConcreteModel()
        model.I = Set(initialize = range(xref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(x))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(y))) # 采用列表初始化产出变量个数的集合
        model.M = Set(initialize = range(len(b))) # 采用列表初始化产出变量个数的集合
        model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
        model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
        model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
        model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
        def objective_rule(model):
            """Return the proper objective function"""
            return model.pomega*1
        def first_rule(model, i):
            """Return the proper input constraint"""
            return -sum(model.px[k] * xref.iloc[i,k] for k in model.K
                    ) + sum(model.py[l] * yref.iloc[i,l] for l in model.L
                    ) - sum(model.pb[m] * bref.iloc[i,m] for m in model.M)    <= 0
        def second_rule(model,k):
            """Return the proper second constraint"""
            return -model.px[k] <= -1 / (len(model.K)*x[k] )
        def third_rule(model,l):
            """Return the proper third constraint"""
            return model.pomega / (len(model.L)+len(model.M)) /y[l]-  model.py[l]<=0
        def forth_rule(model,m):
            """Return the proper forth constraint"""
            return model.pomega / (len(model.L)+len(model.M)) /b[m] - model.pb[m]<=0
        def fifth_rule(model):
            """Return the proper fifth constraint"""
            return model.pomega + sum(model.px[k] * x[k] for k in model.K
                    ) - sum(model.py[l] * y[l] for l in model.L
                    ) + sum(model.pb[m] * b[m] for m in model.M)<=1                           
        model.obj = Objective(rule=objective_rule, sense=maximize, doc='objective function')
        model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
        model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
        model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
        model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')
        model.fifth = Constraint(         rule= fifth_rule , doc='fifth constraint')             
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        if solution.solver.termination_condition == "optimal":     # 终止条件 一般包括三种 optimal, feasible, infeasible
            px[j] = np.asarray(list(model.px[:].value))
            py[j] = np.asarray(list(model.py[:].value))
            pb[j] = np.asarray(list(model.pb[:].value))
            obj[j] = value(model.obj) 
        objdf= pd.DataFrame(obj,index=["obj"]).T
        pxdf = pd.DataFrame(px,index=xcol).T
        pxdf.columns = pxdf.columns.map(lambda x : "shadow price "+ str(x) ) 
        pydf = pd.DataFrame(py,index=ycol).T
        pydf.columns = pydf.columns.map(lambda y : "shadow price "+ str(y) ) 
        pbdf = pd.DataFrame(pb,index=bcol).T
        pbdf.columns = pbdf.columns.map(lambda b : "shadow price "+ str(b) ) 
        pdf=pd.concat([pxdf,pydf],axis=1)
        pdf=pd.concat([pdf,pbdf],axis=1)
        redata = pd.concat([objdf,pdf],axis=1)
    return redata

In [ ]:
import pandas as pd
ex4 = pd.read_stata(r"../../data/Ex4.dta")
data = sbmdual( ex4, ["K","L","Y","CO2"] ,2,1)
data

In [5]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def sbmdual2( dataframe, varname ,P,Q,evaquery,refquery):
    """
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    varname: 投入变量、期望产出变量和非期望产出变量的列表，如["K","L","Y","CO2"]。
    P: 投入变量的个数。
    Q:期望产出变量的个数。
    evaquery: 传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    refquery: 传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    indexlt = dataframe.query(evaquery).index
    indexltref = dataframe.query(refquery).index
    data = dataframe.loc[indexlt,varname]
    dataref = dataframe.loc[indexltref,varname]
    K=P
    L=Q
    M=data.shape[1]-P-Q
    xref=dataref.iloc[:,0:K]
    yref=dataref.iloc[:,K:K+L]
    bref=dataref.iloc[:,K+L:K+L+M]
    xcol=varname[0:K]
    ycol=varname[K:K+L]
    bcol=varname[K+L:K+L+M]
    px,py,pb,obj = {},{},{},{}             
    for j in data.index:
        x=data.loc[j,xcol]
        y=data.loc[j,ycol]
        b=data.loc[j,bcol]
        model = ConcreteModel()
        model.I = Set(initialize = dataref.index) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(x))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(y))) # 采用列表初始化产出变量个数的集合
        model.M = Set(initialize = range(len(b))) # 采用列表初始化产出变量个数的集合
        model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
        model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
        model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
        model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
        def objective_rule(model):
            """Return the proper objective function"""
            return model.pomega*1
        def first_rule(model, i):
            """Return the proper input constraint"""
            return -sum(model.px[k] * xref.iloc[i,k] for k in model.K
                    ) + sum(model.py[l] * yref.iloc[i,l] for l in model.L
                    ) - sum(model.pb[m] * bref.iloc[i,m] for m in model.M)    <= 0
        def second_rule(model,k):
            """Return the proper second constraint"""
            return -model.px[k] <= -1 / (len(model.K)*x[k] )
        def third_rule(model,l):
            """Return the proper third constraint"""
            return model.pomega / (len(model.L)+len(model.M)) /y[l]-  model.py[l]<=0
        def forth_rule(model,m):
            """Return the proper forth constraint"""
            return model.pomega / (len(model.L)+len(model.M)) /b[m] - model.pb[m]<=0
        def fifth_rule(model):
            """Return the proper fifth constraint"""
            return model.pomega + sum(model.px[k] * x[k] for k in model.K
                    ) - sum(model.py[l] * y[l] for l in model.L
                    ) + sum(model.pb[m] * b[m] for m in model.M)<=1                           
        model.obj = Objective(rule=objective_rule, sense=maximize, doc='objective function')
        model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
        model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
        model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
        model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')
        model.fifth = Constraint(         rule= fifth_rule , doc='fifth constraint')             
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        if solution.solver.termination_condition == "optimal":     # 终止条件 一般包括三种 optimal, feasible, infeasible
            px[j] = np.asarray(list(model.px[:].value))
            py[j] = np.asarray(list(model.py[:].value))
            pb[j] = np.asarray(list(model.pb[:].value))
            obj[j] = value(model.obj) 
        objdf= pd.DataFrame(obj,index=["obj"]).T
        pxdf = pd.DataFrame(px,index=xcol).T
        pxdf.columns = pxdf.columns.map(lambda x : "shadow price "+ str(x) ) 
        pydf = pd.DataFrame(py,index=ycol).T
        pydf.columns = pydf.columns.map(lambda y : "shadow price "+ str(y) ) 
        pbdf = pd.DataFrame(pb,index=bcol).T
        pbdf.columns = pbdf.columns.map(lambda b : "shadow price "+ str(b) ) 
        pdf=pd.concat([pxdf,pydf],axis=1)
        pdf=pd.concat([pdf,pbdf],axis=1)
        redata = pd.concat([objdf,pdf],axis=1)
    return redata

In [6]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def sbmdual3( formula, dataframe, evaquery, refquery):
    """
    formula: 产出变量:非期望产出变量~投入变量，如“ Y :CO2  ~ K     L ”
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    evaquery: 传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    refquery: 传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    px,py,pb,obj = {},{},{},{}  
    indexlt = dataframe.query(evaquery).index
    indexltref = dataframe.query(refquery).index
    inputvars = formula.split('~')[1].strip(' ') 
    xcol = re.compile(' +').sub(' ',inputvars).split(' ')
    outputvars = formula.split('~')[0] .split(':')[0] .strip(' ') 
    ycol = re.compile(' +').sub(' ',outputvars) .split(' ')
    unoutputvars = formula.split('~')[0] .split(':')[1] .strip(' ') 
    bcol=re.compile(' +').sub(' ',unoutputvars) .split(' ')   
    data = dataframe.loc[indexlt,xcol+ycol+bcol]
    dataref = dataframe.loc[indexltref,xcol+ycol+bcol]
    xref=dataref.loc[:,xcol]
    yref=dataref.loc[:,ycol]
    bref=dataref.loc[:,bcol]
    for j in data.index:
        x=data.loc[j,xcol]
        y=data.loc[j,ycol]
        b=data.loc[j,bcol]
        model = ConcreteModel()
        model.I = Set(initialize = dataref.index) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(x))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(y))) # 采用列表初始化产出变量个数的集合
        model.M = Set(initialize = range(len(b))) # 采用列表初始化产出变量个数的集合
        model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
        model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
        model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
        model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
        def objective_rule(model):
            """Return the proper objective function"""
            return model.pomega*1
        def first_rule(model, i):
            """Return the proper input constraint"""
            return -sum(model.px[k] * xref.iloc[i,k] for k in model.K
                    ) + sum(model.py[l] * yref.iloc[i,l] for l in model.L
                    ) - sum(model.pb[m] * bref.iloc[i,m] for m in model.M)    <= 0
        def second_rule(model,k):
            """Return the proper second constraint"""
            return -model.px[k] <= -1 / (len(model.K)*x[k] )
        def third_rule(model,l):
            """Return the proper third constraint"""
            return model.pomega / (len(model.L)+len(model.M)) /y[l]-  model.py[l]<=0
        def forth_rule(model,m):
            """Return the proper forth constraint"""
            return model.pomega / (len(model.L)+len(model.M)) /b[m] - model.pb[m]<=0
        def fifth_rule(model):
            """Return the proper fifth constraint"""
            return model.pomega + sum(model.px[k] * x[k] for k in model.K
                    ) - sum(model.py[l] * y[l] for l in model.L
                    ) + sum(model.pb[m] * b[m] for m in model.M)<=1                           
        model.obj = Objective(rule=objective_rule, sense=maximize, doc='objective function')
        model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
        model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
        model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
        model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')
        model.fifth = Constraint(         rule= fifth_rule , doc='fifth constraint')             
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        if solution.solver.termination_condition == "optimal":     # 终止条件 一般包括三种 optimal, feasible, infeasible
            px[j] = np.asarray(list(model.px[:].value))
            py[j] = np.asarray(list(model.py[:].value))
            pb[j] = np.asarray(list(model.pb[:].value))
            obj[j] = value(model.obj) 
        objdf= pd.DataFrame(obj,index=["obj"]).T
        pxdf = pd.DataFrame(px,index=xcol).T
        pxdf.columns = pxdf.columns.map(lambda x : "shadow price "+ str(x) ) 
        pydf = pd.DataFrame(py,index=ycol).T
        pydf.columns = pydf.columns.map(lambda y : "shadow price "+ str(y) ) 
        pbdf = pd.DataFrame(pb,index=bcol).T
        pbdf.columns = pbdf.columns.map(lambda b : "shadow price "+ str(b) ) 
        pdf=pd.concat([pxdf,pydf],axis=1)
        pdf=pd.concat([pdf,pbdf],axis=1)
        redata = pd.concat([objdf,pdf],axis=1)
    return redata

In [7]:
import pandas as pd
ex4 = pd.read_stata(r"../../data/Ex4.dta")
data = sbmdual3( " Y:    CO2 ~ K     L ", ex4, "t==[1,2]","t==[1,2,3]")
data

,obj,shadow price K,shadow price L,shadow price Y,shadow price CO2
0,0.292131,0.000011,0.000117,0.000030,3.958566e-10
1,0.282259,0.000009,0.000116,0.000026,3.769903e-10
3,0.710468,0.000020,0.000438,0.000178,1.109105e-08
4,0.827771,0.000019,0.000432,0.000170,1.060347e-08
6,0.500739,0.000011,0.000196,0.000042,1.090623e-09
7,0.493502,0.000009,0.000189,0.000038,9.961991e-10
9,0.234282,0.000035,0.000332,0.000088,7.142683e-10
10,0.220834,0.000028,0.000329,0.000076,6.739539e-10
12,0.635094,0.000006,0.000082,0.000022,6.073986e-10
13,0.651176,0.000006,0.000081,0.000021,6.255712e-10


In [9]:
ex5 = pd.read_stata(r"../../data/Ex5.dta")
data  = ex5[['labor','capital','energy','gdp','co2']]
K=3
L=1
M=1
xref=data.iloc[:,0:K]
yref=data.iloc[:,K:K+L]
bref=data.iloc[:,K+L:K+L+M]
x=data.iloc[0,0:K]
y=data.iloc[0,K:K+L]
b=data.iloc[0,K+L:K+L+M]
xcol=xref.columns
ycol=yref.columns
bcol=bref.columns
gx=[-1]*len(xcol)
gy=[ 1]*len(ycol)
gb=[-1]*len(bcol)
weight=[]
fenmu = 1*int(gx[0]!=0) + 1*int(gy[0]!=0) + 1*int(gb[0]!=0)
for _ in range(len(xcol)):
    weight.append(1/fenmu/len(xcol))
for _ in range(len(ycol)):
    weight.append(1/fenmu/len(ycol))
for _ in range(len(bcol)):
    weight.append(1/fenmu/len(bcol))  
iweight = weight[0:len(xcol)]
oweight = weight[len(xcol):len(xcol)+len(ycol)]
bweight = weight[len(xcol)+len(ycol):len(xcol)+len(ycol)+len(bcol)]
model = ConcreteModel()
model.I = Set(initialize = range(xref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
model.K = Set(initialize = range(len(xcol))) # 采用列表初始化投入变量个数的集合
model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
model.M = Set(initialize = range(len(bcol))) # 采用列表初始化产出变量个数的集合
model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
def objective_rule(model):
    """Return the proper objective function"""
    return sum(model.px[k]*x[k] for k in model.K  
        ) - sum(model.py[l]*y[l] for l in model.L 
        ) + sum(model.pb[j]*b[j] for j in model.M 
        ) + model.pomega*1 
def first_rule(model, i):
    """Return the proper first constraint"""
    return sum(model.px[k] * xref.loc[i,xcol[k]] for k in model.K
        ) - sum(model.py[l] * yref.loc[i,ycol[l]] for l in model.L
        ) + sum(model.pb[m] * bref.loc[i,bcol[m]] for m in model.M
        ) + model.pomega*1    >=0
def second_rule(model, k):
    """Return the proper second constraint"""
    return  -gx[k]*x[k] * model.px[k] >= iweight[k] 
def third_rule(model, l):
    """Return the proper third constraint"""
    return  gy[l]*y[l] * model.py[l] >= oweight[l]  
def forth_rule(model, j):
    """Return the proper forth constraint"""
    return  -gb[j]*b[j] * model.pb[j] >= bweight[j]                           
model.obj = Objective(rule=objective_rule, sense=minimize, doc='objective function')
model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')         
opt = SolverFactory('mosek') # 指定 mosek 作为求解器
solution = opt.solve(model) # 调用求解器求解
px = np.asarray(list(model.px[:].value))
py = np.asarray(list(model.py[:].value))
pb = np.asarray(list(model.pb[:].value))
obj = value(model.obj) 
print("optimum shadow price x: \n {} ".format(px))
print("optimum shadow price y: \n {} ".format(py))
print("optimum shadow price b: \n {} ".format(pb))
print("optimal object: {}".format(obj))

optimum shadow price x: 
 [3.18854166e-03 1.24359363e-05 1.70782526e-03] 
optimum shadow price y: 
 [0.00073271] 
optimum shadow price b: 
 [1.81810579e-05] 
optimal object: 0.2589296357263148


In [10]:
ex5 = pd.read_stata(r"../../data/Ex5.dta")
data  = ex5[['labor','capital','energy','gdp','co2']]
K=3
L=1
M=1
xref=data.iloc[:,0:K]
yref=data.iloc[:,K:K+L]
bref=data.iloc[:,K+L:K+L+M]
xcol=xref.columns
ycol=yref.columns
bcol=bref.columns
gx=[-1]*len(xcol)
gy=[ 1]*len(ycol)
gb=[-1]*len(bcol)
weight=[]
fenmu = 1*int(gx[0]!=0) + 1*int(gy[0]!=0) + 1*int(gb[0]!=0)
for _ in range(len(xcol)):
    weight.append(1/fenmu/len(xcol))
for _ in range(len(ycol)):
    weight.append(1/fenmu/len(ycol))
for _ in range(len(bcol)):
    weight.append(1/fenmu/len(bcol))  
iweight = weight[0:len(xcol)]
oweight = weight[len(xcol):len(xcol)+len(ycol)]
bweight = weight[len(xcol)+len(ycol):len(xcol)+len(ycol)+len(bcol)]
px,py,pb,obj={},{},{},{}
for j in range(data.shape[0]):
    x=data.iloc[j,0:K]
    y=data.iloc[j,K:K+L]
    b=data.iloc[j,K+L:K+L+M]
    model = ConcreteModel()
    model.I = Set(initialize = range(xref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
    model.K = Set(initialize = range(len(xcol))) # 采用列表初始化投入变量个数的集合
    model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
    model.M = Set(initialize = range(len(bcol))) # 采用列表初始化产出变量个数的集合
    model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
    model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
    model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
    model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
    def objective_rule(model):
        """Return the proper objective function"""
        return sum(model.px[k]*x[k] for k in model.K  
            ) - sum(model.py[l]*y[l] for l in model.L 
            ) + sum(model.pb[j]*b[j] for j in model.M 
            ) + model.pomega*1 
    def first_rule(model, i):
        """Return the proper first constraint"""
        return sum(model.px[k] * xref.loc[i,xcol[k]] for k in model.K
            ) - sum(model.py[l] * yref.loc[i,ycol[l]] for l in model.L
            ) + sum(model.pb[m] * bref.loc[i,bcol[m]] for m in model.M
            ) + model.pomega*1    >=0
    def second_rule(model, k):
        """Return the proper second constraint"""
        return  -gx[k]*x[k] * model.px[k] >= iweight[k] 
    def third_rule(model, l):
        """Return the proper third constraint"""
        return  gy[l]*y[l] * model.py[l] >= oweight[l]  
    def forth_rule(model, j):
        """Return the proper forth constraint"""
        return  -gb[j]*b[j] * model.pb[j] >= bweight[j]                           
    model.obj = Objective(rule=objective_rule, sense=minimize, doc='objective function')
    model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
    model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
    model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
    model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')         
    opt = SolverFactory('mosek') # 指定 mosek 作为求解器
    solution = opt.solve(model) # 调用求解器求解
    if solution.solver.termination_condition == "optimal":     # 终止条件 一般包括三种 optimal, feasible, infeasible
        px[j] = np.asarray(list(model.px[:].value))
        py[j] = np.asarray(list(model.py[:].value))
        pb[j] = np.asarray(list(model.pb[:].value))
        obj[j] = value(model.obj) 

In [11]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def nddfdual( dataframe, varname ,P,Q):
    """
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    varname: 投入变量、期望产出变量和非期望产出变量的列表，如["K","L","Y","CO2"]。
    P: 投入变量的个数。
    Q:期望产出变量的个数。
    """
    data  = dataframe.loc[:,varname]
    K=P
    L=Q
    M=data.shape[1]-P-Q
    xref=data.iloc[:,0:K]
    yref=data.iloc[:,K:K+L]
    bref=data.iloc[:,K+L:K+L+M]
    xcol=xref.columns
    ycol=yref.columns
    bcol=bref.columns
    gx=[-1]*len(xcol)
    gy=[ 1]*len(ycol)
    gb=[-1]*len(bcol)
    weight=[]
    fenmu = 1*int(gx[0]!=0) + 1*int(gy[0]!=0) + 1*int(gb[0]!=0)
    for _ in range(len(xcol)):
        weight.append(1/fenmu/len(xcol))
    for _ in range(len(ycol)):
        weight.append(1/fenmu/len(ycol))
    for _ in range(len(bcol)):
        weight.append(1/fenmu/len(bcol))  
    iweight = weight[0:len(xcol)]
    oweight = weight[len(xcol):len(xcol)+len(ycol)]
    bweight = weight[len(xcol)+len(ycol):len(xcol)+len(ycol)+len(bcol)]
    px,py,pb,obj={},{},{},{}
    for j in range(data.shape[0]):
        x=data.iloc[j,0:K]
        y=data.iloc[j,K:K+L]
        b=data.iloc[j,K+L:K+L+M]
        model = ConcreteModel()
        model.I = Set(initialize = range(xref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(xcol))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
        model.M = Set(initialize = range(len(bcol))) # 采用列表初始化产出变量个数的集合
        model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
        model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
        model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
        model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
        def objective_rule(model):
            """Return the proper objective function"""
            return sum(model.px[k]*x[k] for k in model.K  
                ) - sum(model.py[l]*y[l] for l in model.L 
                ) + sum(model.pb[j]*b[j] for j in model.M 
                ) + model.pomega*1 
        def first_rule(model, i):
            """Return the proper first constraint"""
            return sum(model.px[k] * xref.loc[i,xcol[k]] for k in model.K
                ) - sum(model.py[l] * yref.loc[i,ycol[l]] for l in model.L
                ) + sum(model.pb[m] * bref.loc[i,bcol[m]] for m in model.M
                ) + model.pomega*1    >=0
        def second_rule(model, k):
            """Return the proper second constraint"""
            return  -gx[k]*x[k] * model.px[k] >= iweight[k] 
        def third_rule(model, l):
            """Return the proper third constraint"""
            return  gy[l]*y[l] * model.py[l] >= oweight[l]  
        def forth_rule(model, j):
            """Return the proper forth constraint"""
            return  -gb[j]*b[j] * model.pb[j] >= bweight[j]                           
        model.obj = Objective(rule=objective_rule, sense=minimize, doc='objective function')
        model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
        model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
        model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
        model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')         
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        if solution.solver.termination_condition == "optimal":     # 终止条件 一般包括三种 optimal, feasible, infeasible
            px[j] = np.asarray(list(model.px[:].value))
            py[j] = np.asarray(list(model.py[:].value))
            pb[j] = np.asarray(list(model.pb[:].value))
            obj[j] = value(model.obj) 
        objdf= pd.DataFrame(obj,index=["obj"]).T
        pxdf = pd.DataFrame(px,index=xcol).T
        pxdf.columns = pxdf.columns.map(lambda x : "shadow price "+ str(x) ) 
        pydf = pd.DataFrame(py,index=ycol).T
        pydf.columns = pydf.columns.map(lambda y : "shadow price "+ str(y) ) 
        pbdf = pd.DataFrame(pb,index=bcol).T
        pbdf.columns = pbdf.columns.map(lambda b : "shadow price "+ str(b) ) 
        pdf=pd.concat([pxdf,pydf],axis=1)
        pdf=pd.concat([pdf,pbdf],axis=1)
        redata = pd.concat([objdf,pdf],axis=1)
    return redata

In [12]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def nddfdual2(formula, dataframe, gx=None, gy=None, gb=None,weight =None, evaquery=None, refquery=None ):
    """nddf: Non-radial Directional distance function
    	formula: 产出变量:非期望产出变量~投入变量，如“ Y :CO2  ~ K     L ”
        dataframe: 待评价决策单元的投入产出数据框，按照投入和产出排列
        gx (list, optional): 投入方向向量. 默认为 [-1].
        gy (list, optional): 合意产出方向向量. 默认为 [1].
        gb (list, optional): 非合意产出方向向量. 默认为[-1].
        weight(list, optional): weght matrix
        evaquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"。默认为全部
        refquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]。默认为全部
    """
    px,py,pb,obj={},{},{},{}
    if type(evaquery)==type(None):
        indexlt = dataframe.index
    else:
        indexlt = dataframe.query(evaquery).index
    if type(refquery)==type(None):
        indexltref = dataframe. index
    else:
        indexltref = dataframe.query(refquery).index
    inputvars = formula.split('~')[1].strip(' ') 
    xcol = re.compile(' +').sub(' ',inputvars).split(' ')
    outputvars = formula.split('~')[0] .split(':')[0] .strip(' ') 
    ycol = re.compile(' +').sub(' ',outputvars) .split(' ')
    unoutputvars = formula.split('~')[0] .split(':')[1] .strip(' ') 
    bcol=re.compile(' +').sub(' ',unoutputvars) .split(' ')   
    data = dataframe.loc[indexlt,xcol+ycol+bcol]
    dataref = dataframe.loc[indexltref,xcol+ycol+bcol]
    xref=dataref.loc[:,xcol]
    yref=dataref.loc[:,ycol]
    bref=dataref.loc[:,bcol]
    if type(gx)==type(None):
        gx=[-1]*len(xcol)
    if type(gy)==type(None):
        gy=[1]*len(ycol)
    if type(gb)==type(None):
        gb=[-1]*len(bcol)
    if type(weight) == type(None):
        weight=[]
        fenmu = 1*int(gx[0]!=0) + 1*int(gy[0]!=0) + 1*int(gb[0]!=0)
        for _ in range(len(xcol)):
            weight.append(1/fenmu/len(xcol))
        for _ in range(len(ycol)):
            weight.append(1/fenmu/len(ycol))
        for _ in range(len(bcol)):
            weight.append(1/fenmu/len(bcol))  
    iweight = weight[0:len(xcol)]
    oweight = weight[len(xcol):len(xcol)+len(ycol)]
    bweight = weight[len(xcol)+len(ycol):len(xcol)+len(ycol)+len(bcol)]
    for j in data.index:              # 在data的索引上循环
        x=data.loc[j,xcol]
        y=data.loc[j,ycol]
        b=data.loc[j,bcol]
        model = ConcreteModel()
        model.I = Set(initialize = range(xref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(xcol))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
        model.M = Set(initialize = range(len(bcol))) # 采用列表初始化产出变量个数的集合
        model.px = Var(model.K, bounds=(0, None), within=Reals, doc='shadow price of x')
        model.py = Var(model.L, bounds=(0, None), within=Reals, doc='shadow price of y')
        model.pb = Var(model.M, bounds=(0, None), within=Reals,doc='shadow price of b')
        model.pomega = Var( bounds=(0, None), within=Reals,doc='shadow price of 1')
        def objective_rule(model):
            """Return the proper objective function"""
            return sum(model.px[k]*x[k] for k in model.K  
                ) - sum(model.py[l]*y[l] for l in model.L 
                ) + sum(model.pb[j]*b[j] for j in model.M 
                ) + model.pomega*1 
        def first_rule(model, i):
            """Return the proper first constraint"""
            return sum(model.px[k] * xref.loc[i,xcol[k]] for k in model.K
                ) - sum(model.py[l] * yref.loc[i,ycol[l]] for l in model.L
                ) + sum(model.pb[m] * bref.loc[i,bcol[m]] for m in model.M
                ) + model.pomega*1    >=0
        def second_rule(model, k):
            """Return the proper second constraint"""
            return  -gx[k]*x[k] * model.px[k] >= iweight[k] 
        def third_rule(model, l):
            """Return the proper third constraint"""
            return  gy[l]*y[l] * model.py[l] >= oweight[l]  
        def forth_rule(model, j):
            """Return the proper forth constraint"""
            return  -gb[j]*b[j] * model.pb[j] >= bweight[j]                           
        model.obj = Objective(rule=objective_rule, sense=minimize, doc='objective function')
        model.first = Constraint(model.I, rule= first_rule, doc='first constraint')
        model.second =Constraint(model.K, rule= second_rule , doc='second constraint')
        model.third = Constraint(model.L, rule= third_rule , doc='third constraint')
        model.forth = Constraint(model.M, rule= forth_rule , doc='forth constraint')         
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        if solution.solver.termination_condition == "optimal":     # 终止条件 一般包括三种 optimal, feasible, infeasible
            px[j] = np.asarray(list(model.px[:].value))
            py[j] = np.asarray(list(model.py[:].value))
            pb[j] = np.asarray(list(model.pb[:].value))
            obj[j] = value(model.obj) 
        objdf= pd.DataFrame(obj,index=["obj"]).T
        pxdf = pd.DataFrame(px,index=xcol).T
        pxdf.columns = pxdf.columns.map(lambda x : "shadow price "+ str(x) ) 
        pydf = pd.DataFrame(py,index=ycol).T
        pydf.columns = pydf.columns.map(lambda y : "shadow price "+ str(y) ) 
        pbdf = pd.DataFrame(pb,index=bcol).T
        pbdf.columns = pbdf.columns.map(lambda b : "shadow price "+ str(b) ) 
        pdf=pd.concat([pxdf,pydf],axis=1)
        pdf=pd.concat([pdf,pbdf],axis=1)
        redata = pd.concat([objdf,pdf],axis=1)
    return redata

In [ ]:
import pandas as pd
ex5 = pd.read_stata(r"../../data/Ex5.dta")
data = nddfdual2(' gdp : co2~labor capital energy ', ex5, gx=[-1,-1,-1],gy=[1],gb=[-1],weight=None, )
data